# Análisis de Microbiomas en R

Este tutorial ofrece una visión integral de las principales funciones de la librería [microeco](https://github.com/ChiLiubio/microeco). Este paquete de R está diseñado para asistir al usuario en el análisis de datos de microbiomas utilizando un rango amplio de metodologías que facilitan la experiencia del análisis.  

A lo largo del tutorial, aprenderá a cargar datos en R, manipularlos y normalizarlos, calcular abundancias relativas, realizar análisis descriptivos basados en dichas abundancias, llevar a cabo análisis de diversidad (índices alfa y beta) y representar los resultados de forma gráfica.

### Configuración

In [ ]:
# Descargar archivo r2u 
download.file("https://github.com/eddelbuettel/r2u/raw/master/inst/scripts/add_cranapt_jammy.sh",
              "add_cranapt_jammy.sh")
Sys.chmod("add_cranapt_jammy.sh", "0755")
system("./add_cranapt_jammy.sh")
bspm::enable()
options(bspm.version.check=FALSE)
system("rm add_cranapt_jammy.sh")

In [ ]:
# Función shell_call
shell_call <- function(command, ...) {
  result <- system(command, intern = TRUE, ...)
  cat(paste0(result, collapse = "\n"))
}

> **Recuerde**: Esta configuración solo es necesaria en Google Colab. NO usar en R o Rstudio.

## 1. Instalación de paquetes 

Antes de proceder con la instalación, es importante señalar que, al igual que muchas otras librerías de R, **microeco** es un paquete que se compone por diversas dependencias. Estas dependencias no son más que otras librerías de R que deben instalarse previamente para poder utilizar microeco de una forma adecuada.

El comando `install.packages("BiocManager")` en R se utiliza para instalar el paquete [BiocManager](https://www.bioconductor.org/install/) desde CRAN. Este paquete es una herramienta esencial para trabajar con paquetes del proyecto [Bioconductor](https://www.bioconductor.org/), que proporciona herramientas bioinformáticas para el análisis y comprensión de datos biológicos. Bioconductor es un repositorio especializado en herramientas bioinformáticas y estadísticas diseñadas para el análisis de datos genómicos, transcriptómicos, proteómicos, y más.

In [ ]:
# Instalar BiocManager

install.packages("BiocManager")

In [ ]:
# Instalar paquetes con BiocManager

BiocManager::install("ggtree")
BiocManager::install("metagenomeSeq")
BiocManager::install("ALDEx2")
BiocManager::install("ANCOMBC")
install.packages("file2meco", repos = BiocManager::repositories())
install.packages("MicrobiomeStat", repos = BiocManager::repositories())
install.packages("WGCNA", repos = BiocManager::repositories())
install.packages("remotes")
install.packages("devtools")
install.packages("picante")
install.packages("ggpubr")

In [ ]:
# Instalar paquetes

install.packages("file2meco", repos = BiocManager::repositories())
install.packages("MicrobiomeStat", repos = BiocManager::repositories())
install.packages("WGCNA", repos = BiocManager::repositories())
install.packages("remotes")
install.packages("devtools")
remotes::install_github("jbisanz/qiime2R")

> **Nota:** Es común que las librerías de R sufran actualizaciones constantemente, por lo tanto, a la hora accesar esta guía podrían existir cambios que no estén contemplados en la misma. Se recomienda visitar la siguiente página en caso de que ocurra algún inconveniente https://chiliubio.github.io/microeco_tutorial/ así como https://github.com/ChiLiubio/microeco para la instalación de la librería.

Finalmente se realiza la instalación del paquete *microeco*

In [ ]:
# Instalar microeco paquetes directamente desde un repositorio de GitHub

devtools::install_github("ChiLiubio/microeco")

Una vez descargados los paquetes o librerías, iniciaremos las librerías utilizando la función `library()`

In [ ]:
# Cargar librerías

library(microeco)
library(qiime2R)
library(ggplot2)
library(magrittr)
library(RColorBrewer)
library(ggalluvial)
library(picante)

## 2. Descargar datos

Los datos utilizados en este tutorial se encuentra en un repositorio en Zenodo: https://zenodo.org/records/13972776.

Para descargarlos, utilizaremos la función de R `download.file()`

In [ ]:
urls <- c(
  "https://zenodo.org/records/13972776/files/classification.qza",
  "https://zenodo.org/records/13972776/files/sample-metadata.tsv",
  "https://zenodo.org/records/13972776/files/table.qza",
  "https://zenodo.org/records/13972776/files/rooted_tree.qza"
)

# Nombrar archivos
destfiles <- c("classification.qza", "sample-metadata.tsv", "table.qza", "rooted_tree.qza")

# Descargar los archivos
for (i in seq_along(urls)) {
  download.file(url = urls[i], destfile = destfiles[i], mode = "wb")
  cat(sprintf("Archivo %s descargado correctamente.\n", destfiles[i]))
}

Obtendrá el siguiente resultado:

![descarga](./Images/descarga.png)

## 3. Cargar datos

Para cargar los datos, utilizaremos una librería implementada en R para utilizar los archivos producidos por el programa [QIIME2](https://qiime2.org/). Esta librería lleva el nombre de **qiime2R** y ya fue descargada y cargada en los pasos previos. 

Seguidamente se debe utilizar la función `setwd()` para seleccionar el directorio donde se encuentran los archivos de datos, los cuales están disponibles en la carpeta del tutorial.

In [ ]:
# Seleccionar el directorio

setwd("/content")

In [ ]:
# Cargar datos

datos= qza_to_phyloseq(features = "table.qza",
  taxonomy =   "classification.qza",
  tree = "rooted_tree.qza",
  metadata = "sample-metadata.tsv")

Extraemos cada uno de los archivos de datos para implementarlos en las funciones de *microeco*.

In [ ]:
otu_table = as.data.frame(datos@otu_table)
metadatos = data.frame(datos@sam_data)
arbol = datos@phy_tree
taxonomia = data.frame(datos@tax_table)

Utilizando la función `microtable$new` se puede generar un nuevo conjunto de datos con los archivos cargados. La diferencia principal radica en que este objeto sí es apto para ser utilizado con las demás funciones de la librería.

In [ ]:
# Archivo de datos microtable (microeco)
dataset <-
  microtable$new(
    otu_table = otu_table,
    sample_table = metadatos,
    phylo_tree = arbol,
    tax_table = taxonomia

## 4. Rarefacción

Un término común en el análisis de datos de microbiomas es el de *rarefacción*, este implica un proceso de muestreo aleatorio sobre las unidades contenidas en los OTUS en el cual se debe definir un valor para realizar el muestreo y estandarizar las unidades. Usualmente el valor seleccionado es el de la muestra que contiene la menor cantidad de secuencias. Para verificar el rango de secuencias de los OTUS se utiliza la primera línea de código, la cual indica que la muestra con el menor número de secuencias contiene 1067, por este motivo se ejecuta la rarefacción fijando este valor y utilizando la función `rarefy_samples`. Al ser un muestreo aleatorio se debe fijar una semilla para poder replicar los resultados.

In [ ]:
# Fijamos una semilla para replicar los resultados

set.seed(1)

In [ ]:
# Verificación del rango de valores de los OTU

dataset$sample_sums() %>% range

In [ ]:
# Rarefacción

dataset$rarefy_samples(sample.size = 1067)

> **Nota:** La aplicación de la técnica de rarefacción ha sido custionada a lo largo de los años en el estudio de microbiomas. Para comprender más a profundidad sobre esta discusión se recomienda la siguiente lectura: https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1003531

## 4. Cálculo de abundancia relativa

Una de las características más importantes de los análisis de microbiomas es identificar las principales bacterias que están presentes en las unidades de estudio, para esto se debe utilizar la abundancia relativa, la cual indica la presencia en una escala de 0 a 1 de cada bacteria en cada unidad de estudio.
La siguiente función permite calcular la abundancia en los distintos niveles de la filogenia. Esta abundancia se podrá acceder en `dataset$taxa_abund`. Agregando otro `$` después de `taxa_abund` se puede accesar a la abundancia en diversos niveles taxonómicos como se puede ver en la segunda línea de código.

In [ ]:
# Cálculo de abundancia

dataset$cal_abund()

In [ ]:
# Acceso a los datos
# Se utiliza [1:2,1:2] para mostrar las 2 primeras ranas y bacterias

dataset$taxa_abund$Family[1:2,1:2]

## 5. Funciones de manipulación de datos

### 5.1 Agrupación de taxonomía

En tutoriales anteriores los OTUS se podían agrupar en diversos niveles de la filogenia, sin embargo, había que utilizar diversas librerías para lograr esto. La librería *microtable* permite segregar los OTUS mediante la función `merge_taxa`, esto hará que la información contenida dentro de la tabla de OTUS se ajuste según el nivel de taxonomía (taxa) deseado. Por ejemplo: Kingdom, Phylum, Class, Order, Family o Genus

In [ ]:
gen = dataset$merge_taxa(taxa = "Genus")
gen

Recuerde que la cantidad de OTUS al estar agrupada se reduce con respecto a los presentes en dataset.

### 5.2 Clonación y modificación de datos

En ocasiones se necesitan realizar transformaciones dentro de un archivo de datos, como la que se hizo cuando se creó el objeto gen, sin embargo, siempre es importante contar con el archivo de datos original para mantener un orden a la hora de programar.

Para esto *microeco* implementa la función clone. En el siguiente ejemplo primero se clona el set de datos y después se filtra por una columna de los metadatos, mientras que en el segundo ejemplo se filtra por una bacteria en específico del conjunto de datos de taxonomía.

#### 5.2.1 Clonación y subset por estado del desarrollo

In [ ]:
#Clonamos el dataset original

renacuajos = clone(dataset)

In [ ]:
#Seleccionamos solo los renacuajos de los metadatos

renacuajos$sample_table <- subset(renacuajos$sample_table, life_stage == "tadpole")

In [ ]:
#Modificamos todos los otros archivos para que solo contengan información de los renacuajos
renacuajos$tidy_dataset()

#### 5.2.2 Clonación y subset por bacterias del filo Proteobacteria

In [ ]:
#Clonamos el dataset original
#También se puede filtrar por bacteria

proteo = clone(dataset)

In [ ]:
#Filtramos por proteobacterias

proteo$tax_table <- subset(proteo$tax_table, Phylum == "Proteobacteria")

In [ ]:
#Modificamos todos los otros archivos para que solo contengan información de proteobacterias

proteo$tidy_dataset()

> **Nota importante:** Tenga presente que al modificar la tabla de OTUS o el árbol de la filogenia, las estimaciones de diversidad y abundancia deben realizarse nuevamente. Esto se debe a que los valores con los cuales se estimó la diversidad en dataset no serán los mismos cuando los OTUS o la filogenia se modifican, como en el caso de proteo. Considerando lo anterior debe utilizarse `proteo$cal_albund()` y las demás funciones para volver a estimar los índices de diversidad.

### 5.3 Filtrar por abundancia relativa

Es común que los datos de secuenciación contengan múltiples bacterias que están subrepresentadas en la unidad estudio. A la hora de realizar análisis estadísticos podría ser necesario seleccionar aquellas que presenten una mayor abundancia con respecto al resto de bacterias, para esto se utiliza la función `$filter_taxa` donde se debe de ajustar el límite de abundancia relativa deseado, en ese caso se filtran o eliminan los OTUS que tienen abundancias menores al 0.1%

In [ ]:
#Clonamos el set de datos

dataset_filter <- clone(dataset)

In [ ]:
#Ajustamos el límite de abundancia relativa

dataset_filter$filter_taxa(rel_abund = 0.001)

In [ ]:
#Adaptamos los demás archivos al cambio

dataset_filter$tidy_dataset()

## 6. Análisis descriptivos a partir de la abundancia relativa

Uno de los aspectos más importantes del análisis de datos son los gráficos. Independientemente del campo de estudio, en la mayoría de los casos en los que se cuenta con datos, su presentación usualmente se realiza mediante gráficos de barras, diagramas de puntos, entre otros. Para este caso específico, los gráficos de abundancia relativa, boxplots y mapas de calor son los más comunes a la hora de representar la composición bacteriana. En esta sección se aborda la creación de estos gráficos utilizando *microeco*.

### 6.1 Objeto para la creación de gráficos

Un paso fundamental para crear gráficos de una forma sencilla implica la creación de un objeto apto para la interpretación y ajuste de las características de los gráficos, para ello, se utiliza la función `trans_abund$new`. Esta función, además de modificar la estructura inicial, requiere dos parámetros. El primero, denominado `taxrank`, tiene como función seleccionar el nivel deseado en la taxonomía. Por último, se puede ajustar el número de “taxa” `ntaxa`, que determina cuántos taxa se van a desplegar en los gráficos por orden de abundancia.

In [ ]:
# Visualización  a nivel de Phylum

t1 <- trans_abund$new(dataset = dataset, taxrank = "Phylum", ntaxa = 8)

### 6.2 Gráfico de abundancia relativa por unidad de estudio

Posterior a la creación del objeto t1 se debe utilizar la función `plot_bar` para poder graficar la abundancia relativa de cada sujeto. Otras características de la función son el ajuste de parámetros como `facet` con el fin de agrupar a los individuos según la columna deseada de los metadatos, así como el color en el que se deben desplegar las bacterias que se categorizaron como “otro”, esto mediante el parámetro `others_color`. Finalmente se pueden realizar otras modificaciones como el ajuste de la leyenda mediante `xtext_keep`, entre otros ajustes de formato disponibles.

In [ ]:
t1$plot_bar( facet = "life_stage",others_color = "grey",xtext_keep = F,legend_text_italic = F)

Obtendrá el siguiente resultado: 

![barplot](./Images/barplot.png)

Así como el gráfico de barras convencional, existen otras formas de presentar la misma información, donde nuevamente se observa que los renacuajos presentan una tendencia hacia una composición poco definida a nivel de filo, con respecto a las ranas juvenile y adultas.

In [ ]:
t1$plot_bar(bar_type = "notfull", use_alluvium = TRUE, xtext_angle = 30, xtext_size = 8, color_values = RColorBrewer::brewer.pal(8, "Set2"))

Obtendrá el siguiente resultado:

![barplot](./Images/alluvium.png)

### 6.3 Gráfico de abundancia relativa por grupo de estudio

Aunque presentar la información por individuo es importante, en ciertos casos puede resultar más práctico resumirla según el grupo de estudio. Para lograr esto, se puede realizar una ligera modificación en el objeto original t1 mediante el parámetro `groupmean`. Esto permite obtener el promedio por grupo basándose en alguna de las columnas disponibles en los metadatos.

In [ ]:
t2 = trans_abund$new(dataset = dataset, taxrank = "Phylum", ntaxa = 10, groupmean = "life_stage")
t2$plot_bar(others_color = "grey70", legend_text_italic = FALSE)

Obtendrá el siguiente resultado:

![barplot](./Images/phyllum.png)

### 6.4 Boxplot de abundancia

Para este ejemplo en vez de utilizar el nivel Phylum para presentar la abundancia relativa, se modifica el parámetro `tax_rank` para poder visualizar a nivel de Clase. Además, se mantienen los 4 taxa más abundantes en el gráfico mediante `ntaxa`.

In [ ]:
# Visualización a nivel de clase

t2 = trans_abund$new(dataset = dataset, taxrank = "Class", ntaxa =4)
t2$plot_box(group = "life_stage")

Obtendrá el siguiente resultado:

![barplot](./Images/boxplot.png)

### 6.5 Mapa de calor

Los mapas de calor son una forma útil para visualizar patrones en grandes cantidades de datos, en este caso se seleccionan los 40 taxa más abundantes a nivel de Orden y se despliega la información según los 3 estados del desarrollo de cada rana.

In [ ]:
# Visualización a nivel de Orden

t1 <- trans_abund$new(dataset = dataset, taxrank = "Order", ntaxa = 40)
t1$plot_heatmap(facet = "life_stage", xtext_keep = FALSE, withmargin = FALSE)

Obtendrá el siguiente resultado:

![barplot](./Images/heatmap.png)

Como conclusión del gráfico anterior, se puede extraer que, por ejemplo, las Fusobacteriales presentaron abundancias bajas en las ranas adulto y juvenile, sin embargo, en los renacuajos esta proporción fue mayor.

### 6.6 Gráficos de Pie 

La última visualización de la abundancia relativa para esta guía se implementa mediante el uso de gráficos de dona, en este caso a nivel de Clase. El parámetro `add_level` de la función `plot_pie` permite añadir estos porcentajes al gráfico.

In [ ]:
#Visualización a nivel de Clase

t1 <- trans_abund$new(dataset = dataset, taxrank = "Class", ntaxa = 5, groupmean = "life_stage")

In [ ]:
t1$plot_pie(facet_nrow = 1, add_label = TRUE)

Obtendrá el siguiente resultado:

![barplot](./Images/piechart.png)

## 7. Análisis de diversidad

En el contexto del análisis de la composición bacteriana, se han creado distintas técnicas para cuantificar y analizar la variabilidad de especies bacterianas presentes en una unidad de estudio específica, estas son conocidas como índices de diversidad. La finalidad de esta cuantificación es servir como una herramienta que permita a los investigadores extraer conclusiones acerca de los datos y formular hipótesis sobre los posibles factores que podrían influir en la composición bacteriana.


### 7.1 Diversidad alfa

Los índices más sencillos de analizar son los de la diversidad alfa, ya que estos se componen únicamente de 1 métrica por unidad de estudio. Existen múltiples índices para representar esta diversidad y la función `$cal_alphadiv` permite estimar cada uno de estos. Utilizamos PD = T dentro de la función para incluir la estimación del índice de faith.

In [ ]:
dataset$cal_alphadiv(PD = T)

El acceso a los índices de diversidad se realiza mediante dataset$alpha_diversity. En el código se utiliza la función head() para mostrar los índices de las primeras ranas del dataset

In [ ]:
head(dataset$alpha_diversity)

Obtendrá el siguiente resultado:

En las columnas observará los índices y en las filas las muestras

![barplot](./Images/indices.png)


#### 7.1.1 Análisis de la diversidad Alfa

##### 7.1.1.1 Pruebas de hipótesis

Es común que en el contexto de investigación se plantee estudiar la presencia de diferencias estadísticas entre grupos de estudio, y para esto se han desarrollado diversos métodos con el fin de comparar las distribuciones de los datos y estar a favor o en contra de una hipótesis planteada.
El contenido de este documento no pretende profundizar en el tema de pruebas de hipótesis y se recomienda que el lector indague sobre su correcta ejecución. Por otro lado, para comprender el análisis de la diversidad alfa se plantea el siguiente ejemplo:
Se conoce que podemos calcular la diversidad alfa de cada rana utilizando métricas como el índice de *Shannon*, sin embargo, con el fin de probar si existen diferencias estadísticamente significativas entre los estados del desarrollo, debemos plantear una **hipótesis nula (H0)** y una **alternativa (H1)**.

Donde:

- **H0:** No existen diferencias en el índice de Shannon según los estados de adulto, juvenile y renacuajo.

- **H1:** Existen diferencias en el índice de Shannon en almenos un par de estados (adulto-juvenile-renacuajo).

Para contrastar esta prueba de hipótesis al contar con 3 grupos se debe de utilizar la prueba *Kruskall Wallis*, en caso de contar con 2 grupos se utilizaría una prueba de *Wilcoxon* y dependiendo del caso se podría considerar utilizar el análisis de variancia **(ANOVA)**.

In [ ]:
# Forma usual de probar la hipótesis

kruskal.test(dataset$alpha_diversity$Shannon,g = dataset$sample_table$life_stage)

In [ ]:
# Opción rápida mediante trans_alpha estableciendo el grupo de los metadatos

t1 <- trans_alpha$new(dataset = dataset, group = "life_stage")
t1$cal_diff()
t1$res_diff

# Note que para el índice de shannon el "P.unadj" es el mismo

> **Nota:** En caso de existir diferencias, estas se observarían cuando el P.adj es menor que 0.05, sin embargo, se concluye que no hay suficiente evidencia estadística como para rechazar la hipótesis nula que no existen diferencias en el índice de Shannon entre los estados de adulto, juvenile y renacuajo.

#### 7.1.1.2 Gráficos de los índices de diversidad alfa

Con el fin de presentar los hallazgos encontrados en la sección anterior, la librería ofrece gráficos para el índice de diversidad según los grupos de estudio, para esto debemos haber utilizado la función `cal_diff()` en nuestro set de datos para así incorporar la función `plot_alpha`. Ajustando el parámetro measure se puede seleccionar el índice de diversidad de interés y el parámetro shape ajusta la forma de los puntos de cada grupo.

In [ ]:
# Gráfico del índice de shannon por etapa de la vida

t1$plot_alpha(measure = "Shannon", shape = "life_stage")

Obtendrá el siguiente resultado:


![barplot](./Images/alpha.png)

> **Nota:** Para modificar el "ns" que aparece en la esquina izquierda del gráfico consulte el link proporcionado al inicio de la guía. Considerando lo propuesto en esta sección, se tienen las herramientas necesarias para analizar la diversidad alfa mediante pruebas de hipótesis y reportar las diferencias utilizando visualizaciones como los boxplots.

### 7.2 Diversidad beta

El proceso de estimación de la diversidad beta es muy similar, en este caso se debe nombrar al archivo de datos y posteriormente utilizar `$cal_betadiv`

In [ ]:
dataset$cal_betadiv()

In [ ]:
head(data.frame(dataset$beta_diversity))[1:3,1:3]

> **Nota:** Para cálculo de estos índices microeco se beneficia de otras librerías como *vegan* o *physeq*, es importante que el usuario haga uso de la guía teniendo en cuenta que la estimación de estas métricas se realiza utilizando diversas fórmulas y procesos matemáticos los cuales eventualmente podrían ajustarse según el tipo de datos y la cantidad con la que se cuente. Por default la función utiliza el método bray y jaccard como métricas de diversidad beta. A pesar de que no aparezca explícitamente en la función, esta se está alimentando del siguiente parámetro method = c("bray","jaccard"). Consulte la ayuda de la función vegdist para ver las opciones de method que se pueden utilizar.

#### 7.2.1 Análisis de la diversidad beta

Hasta ahora hemos calculado índices de diversidad alfa y beta, sin embargo, la medida para representar la diversidad beta no es un valor que pertenezca a una unidad, como lo fue en el caso de la diversidad alfa. En este caso se utiliza la matriz de OTUS para calcular distancias entre cada par de unidades del estudio, donde el objetivo es representar la variabilidad o disimilitud entre cada par de muestras con la finalidad de utilizar herramientas de aprendizaje automático que ayuden a obtener visualizaciones o pruebas estadísticas como PERMANOVA (ver https://uw.pressbooks.pub/appliedmultivariatestatistics/chapter/permanova/) que permitan obtener conclusiones a partir de los datos.
Como en casos anteriores, primero se debe crear un objeto que permita interpretar la diversidad beta, para esto se utiliza la función trans_beta$new. Se debe ajustar el grupo presente en los metadatos, así como el parámetro measure que debe de coincidir con el method implementado en la función $cal_betadiv()(anteriormente se utilizó bray y jaccard como method).

In [ ]:
# Objeto de interpretación de diversidad, measure debe ser bray o jaccard
t1 <- trans_beta$new(dataset = dataset, group = "life_stage", measure = "bray")

##### 7.2.1.1 Representación gráfica de la diversidad beta

Para representar gráficamente la diversidad beta existen diversos métodos como el PCoA, PCA o el escalamiento no métrico multidimensional (NMDS), la función `$cal_ordination` permite seleccionar el tipo de visualización según las necesidades del usuario mediante el parámetro ordination.

In [ ]:
# Establecemos una semilla para que los resultados coincidan

set.seed(1)

In [ ]:
# Estimación de coordanadas para representar la beta diversidad

t1$cal_ordination(method = "NMDS")

In [ ]:
# Gráfico de NMDS

t1$plot_ordination(plot_color = "life_stage",plot_shape = "life_stage", plot_type = c("point", "ellipse"))

Obtendrá el siguiente resultado:

![barplot](./Images/beta.png)

> **Nota:** Al utilizar `cal_ordination` la función da un aviso de que no hay suficientes datos, dando indicios de que la representación gráfica no es fiel. Se recomienda visitar https://uw.pressbooks.pub/appliedmultivariatestatistics/chapter/nmds/ para comprender el paso a paso de esta técnica.

> **¡Tener en cuenta!:** Las distancias de Bray-curtis son calculadas a partir de la matriz de OTUS que se puede accesar utilizando la función `dataset$otu_table`. Note que si se modifica esta matriz porque se creó un subset de los datos o porque estos se agruparon según un nivel taxonómico específico, la distancia entre las muestras cambia y con ella las visualizaciones de la diversidad beta.

In [ ]:
set.seed(1)
prueba = clone(dataset)
filo = prueba$merge_taxa(taxa = "Phylum") # Se agrupa por filo
filo$cal_betadiv() # Se calcula la beta diversidad

t1$plot_ordination(plot_color = "life_stage",plot_shape = "life_stage", plot_type = c("point", "ellipse"))

Obtendrá el siguiente resultado:

![barplot](./Images/beta2.png)